In [1]:
pip install transformers

     |████████████████████████████████| 2.6 MB 8.3 MB/s 
     |████████████████████████████████| 895 kB 36.2 MB/s 
     |████████████████████████████████| 3.3 MB 38.2 MB/s 
     |████████████████████████████████| 636 kB 44.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("sismetanin/rubert-toxic-pikabu-2ch")

model = AutoModelForSequenceClassification.from_pretrained("sismetanin/rubert-toxic-pikabu-2ch")

In [3]:
import pandas as pd
xls = pd.ExcelFile('Негатив_пример.xlsx')
xls_toxic = pd.read_excel('Негатив_пример.xlsx', index_col=None, header=0)  

In [4]:
xls_toxic['Text'][3].split(' ')

['завтра', 'же', 'нахер', 'все', 'позакрою', 'тут,', 'ебучее', 'говно']

In [20]:
def razbienie(str):
    spisok_str = str.split(' ')

    new_spisok = []
    i = 0
    if len(spisok_str) == 1:
        new_spisok.append(spisok_str[i])
        return new_spisok
    while i <= len(spisok_str)-1:
        if i == 0:
          new_spisok.append(spisok_str[i-1] + ' ' + spisok_str[i] + ' ' + spisok_str[i+1])
          i += 1
        elif i == len(spisok_str)-1 and i != 0:
          new_spisok.append(spisok_str[i-1] + ' ' + spisok_str[i])
          i += 1
        else:
          new_spisok.append(spisok_str[i-1] + ' ' + spisok_str[i] + ' ' + spisok_str[i+1])
  
          i += 1
    return new_spisok

In [23]:
pt_batches = []
for i in range(len(xls_toxic['Text'])):
  #print(xls_toxic['Text'][i].split(' '))
  pt_batch = tokenizer(
    razbienie(xls_toxic['Text'][i]),
    padding=True,    truncation=True,    max_length=512,    return_tensors="pt" )
  pt_batches.append(pt_batch)

In [25]:
from torch import nn
pt_predictions = []
for pt_batch in pt_batches:
  pt_output = model(**pt_batch)
  pt_prediction = nn.functional.softmax(pt_output.logits, dim=-1)
  pt_predictions.append(pt_prediction)

In [28]:
pt_predictions[1]

tensor([[0.9979, 0.0021],
        [0.9970, 0.0030]], grad_fn=<SoftmaxBackward>)

In [29]:
xls_toxic['Text'][3], pt_predictions[3]

('завтра же нахер все позакрою тут, ебучее говно', tensor([[0.0564, 0.9436],
         [0.0126, 0.9874],
         [0.0236, 0.9764],
         [0.3741, 0.6259],
         [0.9895, 0.0105],
         [0.0247, 0.9753],
         [0.0195, 0.9805],
         [0.0412, 0.9588]], grad_fn=<SoftmaxBackward>))

In [34]:
import numpy as np
bad_words_full = []
procents = []
for i in range(len(pt_predictions)):
  bad_words = []
  words = razbienie(xls_toxic['Text'][i])
  #print(float(pt_predictions[i][0][1])) 
  for j in range(len(pt_predictions[i])):
    if float(pt_predictions[i][j][1]) > 0.7:
      #print(type(words[j]))
      bad_words.append((words[j], np.round(float(pt_predictions[i][j][1]), 4)))
      #bad_words.append(float(pt_predictions[i][j][1]))
  procent = len(bad_words)/len(words)
  procents.append(procent)
  bad_words_full.append(bad_words)
print(bad_words_full)

[[], [], [('мой деньги паскуды,', 0.9484)], [('говно завтра же', 0.9436), ('завтра же нахер', 0.9874), ('же нахер все', 0.9764), ('позакрою тут, ебучее', 0.9753), ('тут, ебучее говно', 0.9805), ('ебучее говно', 0.9588)], [], [('Говно', 0.9455)], [('что за хрень', 0.9911), ('за хрень творится', 0.967)], [], [('жалобу буду писать', 0.8645), ('буду писать жалобу', 0.881), ('писать жалобу', 0.9057)], [('! Вы издеваетесь', 0.9597), ('Вы издеваетесь ?!', 0.9512), ('издеваетесь ?! Мне', 0.9262), ('и я не', 0.8666)], [], [], [('издеваетесь? вы издеваетесь?', 0.929), ('вы издеваетесь?', 0.9571)], [], [], [('!!! Пидары ебаные', 0.9943), ('Пидары ебаные !!!', 0.9927), ('ебаные !!!', 0.988)], [('Роспотребнадзор я сейчас', 0.867)], [], [], [('ебаные Уроды вы', 0.9893), ('Уроды вы ебаные', 0.9807), ('вы ебаные', 0.9707)], [], [], [('что я их', 0.9845), ('Че за херня', 0.9939), ('за херня происходит', 0.9868), ('херня происходит', 0.8873)], [('сообщить. Вы издевается?', 0.9596), ('Вы издевается? Уже'

In [35]:
xls_toxic['Bad_words'] = bad_words_full
xls_toxic['Percentage of bad'] = np.round(procents, 4)

In [36]:
xls_toxic

,Text,Bad_words,Percentage of bad
0,ужасная поддержка!,[],0.0000
1,ужасная поддержка,[],0.0000
2,"где мой деньги паскуды, сколько можно херней з...","[(мой деньги паскуды,, 0.9484)]",0.1250
3,"завтра же нахер все позакрою тут, ебучее говно","[(говно завтра же, 0.9436), (завтра же нахер, ...",0.7500
4,У вас приложение говно полное,[],0.0000
...,...,...,...
105,"скажите, где и кому сообщать по поводу работы ...","[(никому. у вас, 0.8726)]",0.0500
106,что за хуйня опять у вас там творится,"[(что за хуйня, 0.9947), (за хуйня опять, 0.99...",0.7500
107,потому по щётку такая хуйня,"[(хуйня потому по, 0.9911), (потому по щётку, ...",0.8000
108,"что значит ""зафиксировать обращение""? я хочу в...",[],0.0000


In [38]:
xls_toxic.to_excel('негатив_вывод_ver_7.xlsx')